In [53]:
import sys
import os
import pickle
import numpy as np
import pandas as pd
from trodes import read_exported as tre
sys.path.append(r'c:\Users\zhaoz\Desktop\Research\SocialMemory\diff_fam_social_memory_ephys')

In [54]:
def pickle_this(thing_to_pickle, file_name):
    """
    Pickles things
    Args (2):   
        thing_to_pickle: anything you want to pickle
        file_name: str, filename that ends with .pkl 
    Returns:
        none
    """
    with open(file_name,'wb') as file:
        pickle.dump(thing_to_pickle, file)

def unpickle_this(pickle_file):
    """
    Unpickles things
    Args (1):   
        file_name: str, pickle filename that already exists and ends with .pkl
    Returns:
        pickled item
    """
    with open(pickle_file, 'rb') as file:
        return(pickle.load(file))

In [55]:
path = r"C:\Users\zhaoz\Desktop\Research\Cooperation\ecu_behaviors"

box_to_ecu_dict = {
            1: {'dio_ECU_Din1': 'selfish light',
                'dio_ECU_Din2': 'coop light',
                'dio_ECU_Din6': 'selfish nose poke',
                'dio_ECU_Din10': 'coop nose poke',
                'dio_ECU_Din8': 'subject port entry',
                'dio_ECU_Din16': 'recipient port entry'},   
            2: {'dio_ECU_Din3': 'selfish light',
                'dio_ECU_Din4': 'coop light',
                'dio_ECU_Din22': 'selfish nose poke',
                'dio_ECU_Din26': 'coop nose poke',
                'dio_ECU_Din24': 'subject port entry',
                'dio_ECU_Din32': 'recipient port entry'}}

# Read the box distributions for different recordings
df = pd.read_csv(path + "\Coop_ephys_Box_organization(Sheet1).csv")
df["Individual name"] = df["Individual name"].astype(str) + "_merged"
print(df.to_string)

<bound method DataFrame.to_string of                              Individual name         Trainning  Box  \
0   20250517_115803_alternates_D1_1-2_merged  Stage 3 last day  1.0   
1   20250517_115803_alternates_D1_1-3_merged  Stage 3 last day  1.0   
2   20250517_115803_alternates_D1_2-4_merged  Stage 3 last day  2.0   
3   20250517_115803_alternates_D1_2-1_merged  Stage 3 last day  2.0   
4   20250507_103358_alternates_D1_6-1_merged  Stage 3 last day  1.0   
..                                       ...               ...  ...   
82      20250517_134054_Stage4_D8_4-2_merged      Stage4 day 8  2.0   
83      20250517_134054_Stage4_D8_4-3_merged      Stage4 day 8  1.0   
84      20250517_134054_Stage4_D8_4-1_merged      Stage4 day 8  1.0   
85      20250517_121746_Stage4_D8_6-1_merged      Stage4 day 8  2.0   
86      20250517_121746_Stage4_D8_6-3_merged      Stage4 day 8  2.0   

   Mice condition  
0         Subject  
1       Recipient  
2         Subject  
3       Recipient  
4         

In [56]:
def trodes_behaviors_to_timestamps(data, clockrate):
	'''
	Convert the data in .dat file format to normal behavior numpy array
	'''
	behavior_array = []
	event = []
	status = 1

	for (time, value) in data[1:]:
		if value[0] != status:
			raise ValueError('Invalid timestamp - the event has not started/stopped')
		
		event.append(time[0]/int(clockrate))
		if (status == 0):
			behavior_array.append(event)
			event = []
		status = 1 - status
	
	return np.array(behavior_array, dtype=float).tolist()

In [57]:
def load_trodes_recording(relative_path):
	'''
	Read the relative path to the folder that contains merged.DIO folder
	'''
	absolute_path = os.path.join(path, relative_path)
	name = relative_path.split('.')[0]	#20250508_100203_Stage4_D1_1-2_merged
	box = int(df.loc[df["Individual name"] == name, "Box"].iloc[0])	#1 or 2
	behaviors_dict = {}

	# Walk through the .dat files
	for root, dirs, files in os.walk(absolute_path):
		for file in files:
			if file.endswith('.dat'):
				din = file.split('.')[1]
				if din in box_to_ecu_dict[box]:	#if the din channel is meaningful
					data = tre.read_trodes_extracted_data_file(os.path.join(absolute_path, file))
					event = box_to_ecu_dict[box][din]
					behavior_data = data['data']
					# print(event, len(behavior_data))
					# print(behavior_data)
					# print()
					behavior_array = trodes_behaviors_to_timestamps(behavior_data, data['clockrate'])
					behaviors_dict[event] = behavior_array

	return (name, behaviors_dict)

In [58]:
def print_n_o_events(behaviors_dict):
	for key, value in behaviors_dict.items():
		print('#', key, "-", len(value))

In [65]:
# Stage 4 Day 1
stage_4_day_1_behavior_dict = {}
for root, dirs, files in os.walk(path):
	for directory in dirs:
		if directory.endswith("DIO"):
			print(directory)
			rec_name, behaviors = load_trodes_recording(directory)
			stage_4_day_1_behavior_dict[rec_name] = behaviors

pickle_this(stage_4_day_1_behavior_dict, path + "\Stage4_D1.pkl")

20250508_100203_Stage4_D1_1-2_merged.DIO
20250508_100203_Stage4_D1_1-3_merged.DIO
20250508_100203_Stage4_D1_2-1_merged.DIO
20250508_100203_Stage4_D1_2-4_merged.DIO
20250508_112121_Stage4_D1_6-1_merged.DIO
20250508_112121_Stage4_D1_6-3_merged.DIO
20250509_131426_Stage4_D1_4-1_merged.DIO
20250509_131426_Stage4_D1_4-2_merged.DIO
20250509_131426_Stage4_D1_4-3_merged.DIO
20250509_131426_Stage4_D1_4-4_merged.DIO


In [64]:
Stage4_D1_behaviors = unpickle_this(path + "\Stage4_D1.pkl")
print(Stage4_D1_behaviors)

{'20250508_100203_Stage4_D1_1-2_merged': {'selfish light': [[302.23645, 307.2363], [309.27605, 314.27615], [324.36675, 329.36645], [330.85635, 335.8564], [338.06655, 343.0665], [347.0568, 352.0566], [355.44665, 360.4467], [364.84685, 369.84705], [372.30685, 377.307], [381.06695, 386.067], [390.04745, 395.04715], [402.5072, 407.5073], [415.70735, 420.7076], [423.8775, 428.8778], [432.0476, 437.0478], [442.3381, 447.33765], [450.9978, 455.9983], [459.37805, 464.378], [466.81805, 471.8181], [477.23815, 482.23845], [488.9883, 493.98835], [497.87865, 502.87845], [505.1085, 510.10845], [512.6886, 517.68865], [521.4087, 526.409], [528.77895, 533.77885], [536.8489, 541.84895], [544.4391, 549.43905], [557.47915, 562.4796], [564.74925, 569.74945], [572.93935, 577.9395], [581.02945, 586.0295], [608.7398, 613.73985], [616.28, 621.27995], [629.91055, 634.9101], [652.6003, 657.6005], [658.7404, 663.7407], [665.3605, 670.36055], [673.2706, 678.2709], [685.07075, 690.07095], [740.5714, 745.5715], [749